In [ ]:
import momepy
import pandas as pd
from cityseer.metrics import layers
from cityseer.tools import graphs, io
from osmnx import features

In [40]:
lng, lat = -0.13396079424572427, 51.51371088849723
buffer = 1500
poly_wgs, epsg_code = io.buffered_point_poly(lng, lat, buffer)
G = io.osm_graph_from_poly(poly_wgs)
G = graphs.nx_decompose(G, 50)
G_dual = graphs.nx_to_dual(G)
nodes_gdf, _edges_gdf, network_structure = io.network_structure_from_nx(G_dual)

INFO:cityseer.tools.graphs:Generating interpolated edge geometries.
100%|██████████| 23999/23999 [00:00<00:00, 49775.25it/s]
INFO:cityseer.tools.io:Converting networkX graph to CRS code 32630.
INFO:cityseer.tools.io:Processing node x, y coordinates.
100%|██████████| 22280/22280 [00:00<00:00, 363202.48it/s]
INFO:cityseer.tools.io:Processing edge geom coordinates, if present.
100%|██████████| 23999/23999 [00:01<00:00, 13748.75it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 22280/22280 [00:00<00:00, 32127.82it/s]
INFO:cityseer.tools.util:Creating edges STR tree.
100%|██████████| 12113/12113 [00:00<00:00, 13632.40it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 7339/7339 [00:00<00:00, 148250.52it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 8952/8952 [00:00<00:00, 914786.44it/s]
INFO:cityseer.tools.util:Creating edges STR tree.
100%|██████████| 6626/6626 [00:00<00:0

In [41]:
data_gdf = features.features_from_polygon(poly_wgs, tags={"building": True})
data_gdf = data_gdf.to_crs(G.graph["crs"])
data_gdf.tail()

geometry  \
element id                                                              
way     1374498886  POLYGON ((699072.276 5711987.717, 699073.651 5...   
        1374681344  POLYGON ((700334.357 5710842.473, 700337.684 5...   
        1374681345  POLYGON ((700347.126 5710850.783, 700337.684 5...   
        1374681346  POLYGON ((700335.868 5710827.778, 700339.655 5...   
        1374820282  POLYGON ((699547.129 5710318.314, 699559.064 5...   

                   addr:city addr:postcode    addr:street brand  \
element id                                                        
way     1374498886    London      WC1H 0PD  Gordon Square   NaN   
        1374681344       NaN           NaN            NaN   NaN   
        1374681345       NaN           NaN            NaN   NaN   
        1374681346       NaN           NaN            NaN   NaN   
        1374820282       NaN           NaN            NaN   NaN   

                   brand:wikidata brand:wikipedia    building email  \
element id                                                            
way     1374498886            NaN             NaN       kiosk   NaN   
        1374681344            NaN             NaN  commercial   NaN   
        1374681345            NaN             NaN         yes   NaN   
        1374681346            NaN             NaN  commercial   NaN   
        1374820282            NaN             NaN         yes   NaN   

                            name  ... name:th name:tt name:ur name:vec  \
element id                        ...                                    
way     1374498886  Garden Kiosk  ...     NaN     NaN     NaN      NaN   
        1374681344           NaN  ...     NaN     NaN     NaN      NaN   
        1374681345           NaN  ...     NaN     NaN     NaN      NaN   
        1374681346           NaN  ...     NaN     NaN     NaN      NaN   
        1374820282           NaN  ...     NaN     NaN     NaN      NaN   

                   name:vi name:war name:wuu name:yi name:yue building:flats  
element id                                                                    
way     1374498886     NaN      NaN      NaN     NaN      NaN            NaN  
        1374681344     NaN      NaN      NaN     NaN      NaN            NaN  
        1374681345     NaN      NaN      NaN     NaN      NaN            NaN  
        1374681346     NaN      NaN      NaN     NaN      NaN            NaN  
        1374820282     NaN      NaN      NaN     NaN      NaN            NaN  

[5 rows x 554 columns]

In [42]:
bldg_way = data_gdf.loc["way"]
bldg_way = bldg_way.reset_index(level=0, drop=True)
bldg_way = bldg_way[["geometry"]]
bldg_way.head()

,geometry
0,"POLYGON ((699578.37 5711981.881, 699573.237 57..."
1,"POLYGON ((699509.329 5711958.943, 699502.917 5..."
2,"POLYGON ((699771.225 5711711.453, 699763.808 5..."
3,"POLYGON ((699408.814 5712013.541, 699453.773 5..."
4,"POLYGON ((698517.969 5709739.367, 698517.359 5..."


In [43]:
bldg_rel = data_gdf.loc["relation"]
bldg_rel = bldg_rel.reset_index(level=0, drop=True)
bldg_rel = bldg_rel[["geometry"]]
bldg_rel.head()

,geometry
0,"POLYGON ((698284.165 5711770.625, 698310.212 5..."
1,"POLYGON ((699365.964 5709570.204, 699366.296 5..."
2,"POLYGON ((699421.233 5709690.565, 699431.714 5..."
3,"POLYGON ((699578.614 5709619.103, 699586.076 5..."
4,"POLYGON ((699708.798 5710453.42, 699717.241 57..."


In [44]:
bldgs_gpd = pd.concat([bldg_way, bldg_rel], axis=0)
bldgs_gpd = bldgs_gpd.explode(ignore_index=True)
bldgs_gpd = bldgs_gpd.reset_index(drop=True)
bldgs_gpd.head()

,geometry
0,"POLYGON ((699578.37 5711981.881, 699573.237 57..."
1,"POLYGON ((699509.329 5711958.943, 699502.917 5..."
2,"POLYGON ((699771.225 5711711.453, 699763.808 5..."
3,"POLYGON ((699408.814 5712013.541, 699453.773 5..."
4,"POLYGON ((698517.969 5709739.367, 698517.359 5..."


In [45]:
bldgs_gpd["area"] = bldgs_gpd.geometry.area
bldgs_gpd["perimeter"] = bldgs_gpd.geometry.length
bldgs_gpd["compactness"] = momepy.circular_compactness(bldgs_gpd.geometry)
bldgs_gpd["orientation"] = momepy.orientation(bldgs_gpd.geometry)
bldgs_gpd["shape_index"] = momepy.shape_index(bldgs_gpd.geometry)

In [46]:
bldgs_gpd.geometry.type.unique()

array(['Polygon'], dtype=object)

In [47]:
# compute pub accessibility
distances = [100, 200, 400, 800]
nodes_gdf, bldgs_gpd = layers.compute_stats(
    bldgs_gpd,
    stats_column_labels=[
        "area",
        "perimeter",
        "compactness",
        "orientation",
        "shape_index",
    ],
    nodes_gdf=nodes_gdf,
    network_structure=network_structure,
    distances=distances,
)

INFO:cityseer.metrics.layers:Computing statistics.
INFO:cityseer.metrics.layers:Assigning data to network.
100%|██████████| 4987/4987 [00:02<00:00, 2479.62it/s]
INFO:cityseer.config:Metrics computed for:
INFO:cityseer.config:Distance: 100m, Beta: 0.04, Walking Time: 1.25 minutes.
INFO:cityseer.config:Distance: 200m, Beta: 0.02, Walking Time: 2.5 minutes.
INFO:cityseer.config:Distance: 400m, Beta: 0.01, Walking Time: 5.0 minutes.
INFO:cityseer.config:Distance: 800m, Beta: 0.005, Walking Time: 10.0 minutes.
/Users/gareth/dev/benchmark-urbanism/cityseer-examples/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/Users/gareth/dev/benchmark-urbanism/cityseer-examples/.

In [48]:
nodes_gdf.columns

Index(['ns_node_idx', 'x', 'y', 'live', 'weight', 'primal_edge',
       'primal_edge_node_a', 'primal_edge_node_b', 'primal_edge_idx',
       'dual_node',
       ...
       'cc_shape_index_sum_800_nw', 'cc_shape_index_sum_800_wt',
       'cc_shape_index_mean_800_nw', 'cc_shape_index_mean_800_wt',
       'cc_shape_index_count_800_nw', 'cc_shape_index_count_800_wt',
       'cc_shape_index_var_800_nw', 'cc_shape_index_var_800_wt',
       'cc_shape_index_max_800', 'cc_shape_index_min_800'],
      dtype='object', length=210)